In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

device = torch.device("cpu")
print(f"Using device: {device}")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0)

testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=0)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class RDropLoss(nn.Module):
    def __init__(self, alpha):
        super(RDropLoss, self).__init__()
        self.cross_entropy = nn.CrossEntropyLoss()
        self.alpha = alpha

    def compute_kl_loss(self, p, q):
        p_prob = F.softmax(p, dim=-1)
        q_prob = F.softmax(q, dim=-1)
        p_log_prob = F.log_softmax(p, dim=-1)
        q_log_prob = F.log_softmax(q, dim=-1)
        kl_pq = F.kl_div(p_log_prob, q_prob, reduction='batchmean')
        kl_qp = F.kl_div(q_log_prob, p_prob, reduction='batchmean')
        return (kl_pq + kl_qp) / 2

    def forward(self, p, q, labels):
        ce_loss = (self.cross_entropy(p, labels) + self.cross_entropy(q, labels)) / 2
        kl_loss = self.compute_kl_loss(p, q)
        return ce_loss + self.alpha * kl_loss

def train_baseline(model, trainloader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc=f"Epoch {epoch+1} [Baseline Training]")):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Loss: {running_loss / len(trainloader):.3f}")

def train_rdrop(model, trainloader, criterion_rdrop, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc=f"Epoch {epoch+1} [R-Drop Training]")):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs_1 = model(inputs)
        outputs_2 = model(inputs)
        loss = criterion_rdrop(outputs_1, outputs_2, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Loss: {running_loss / len(trainloader):.3f}")

def evaluate_model(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Accuracy: {accuracy:.2f} %')
    return accuracy

def run_training(num_epochs, learning_rate, alpha=None):
    model = SimpleCNN().to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    start_epoch = 0

    if alpha is not None:
        model_type = f"R-Drop Model {alpha}"
        criterion = RDropLoss(alpha=alpha)
        train_function = train_rdrop
        model_save_path = f'rdrop_model_{alpha}_checkpoint.pth'
    else:
        model_type = "Baseline Model"
        criterion = nn.CrossEntropyLoss()
        train_function = train_baseline
        model_save_path = 'baseline_model_checkpoint.pth'

    print(f"--- Starting {model_type} Training ---")

    try:
        checkpoint = torch.load(model_save_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Loaded {model_type} from checkpoint. Resuming training from epoch {start_epoch}.")
    except FileNotFoundError:
        print(f"No saved {model_type} found. Starting training from scratch.")
    except Exception as e:
        print(f"Error loading {model_type} checkpoint: {e}. Starting training from scratch.")

    for epoch in range(start_epoch, start_epoch + num_epochs):
        train_function(model, trainloader, criterion, optimizer, epoch)
        evaluate_model(model, testloader)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, model_save_path)
        print(f"{model_type} saved at epoch {epoch+1} to {model_save_path}")

    print(f"\n--- Finished {model_type} Training ---\n")

Using device: cpu


100%|██████████| 170M/170M [00:01<00:00, 105MB/s]


In [ ]:
# Baseline Training
run_training(num_epochs=10, learning_rate=0.001)

--- Starting Baseline Model Training ---
Loaded Baseline Model from checkpoint. Resuming training from epoch 20.


Epoch 21 [Baseline Training]: 100%|██████████| 782/782 [01:40<00:00,  7.77it/s]


Loss: 0.909
Accuracy: 74.38 %
Baseline Model saved at epoch 21 to baseline_model_checkpoint.pth


Epoch 22 [Baseline Training]: 100%|██████████| 782/782 [01:40<00:00,  7.77it/s]


Loss: 0.893
Accuracy: 73.88 %
Baseline Model saved at epoch 22 to baseline_model_checkpoint.pth


...

Loss: 0.864
Accuracy: 75.91 %
Baseline Model saved at epoch 28 to baseline_model_checkpoint.pth


Epoch 29 [Baseline Training]: 100%|██████████| 782/782 [01:40<00:00,  7.77it/s]


Loss: 0.870
Accuracy: 75.65 %
Baseline Model saved at epoch 29 to baseline_model_checkpoint.pth


Epoch 30 [Baseline Training]: 100%|██████████| 782/782 [01:40<00:00,  7.75it/s]


Loss: 0.858
Accuracy: 76.02 %
Baseline Model saved at epoch 30 to baseline_model_checkpoint.pth

--- Finished Baseline Model Training ---



In [ ]:
# R-Drop Training (alpha=0.4)
run_training(num_epochs=30, learning_rate=0.001, alpha=0.4)

--- Starting R-Drop Model 0.4 Training ---
No saved R-Drop Model 0.4 found. Starting training from scratch.


Epoch 1 [R-Drop Training]: 100%|██████████| 782/782 [02:52<00:00,  4.53it/s]


Loss: 1.705
Accuracy: 53.89 %
R-Drop Model 0.4 saved at epoch 1 to rdrop_model_0.4_checkpoint.pth


Epoch 2 [R-Drop Training]: 100%|██████████| 782/782 [02:54<00:00,  4.47it/s]


Loss: 1.450
Accuracy: 58.71 %
R-Drop Model 0.4 saved at epoch 2 to rdrop_model_0.4_checkpoint.pth


...

Loss: 0.934
Accuracy: 76.16 %
R-Drop Model 0.4 saved at epoch 28 to rdrop_model_0.4_checkpoint.pth


Epoch 29 [R-Drop Training]: 100%|██████████| 782/782 [02:50<00:00,  4.58it/s]


Loss: 0.930
Accuracy: 74.59 %
R-Drop Model 0.4 saved at epoch 29 to rdrop_model_0.4_checkpoint.pth


Epoch 30 [R-Drop Training]: 100%|██████████| 782/782 [02:49<00:00,  4.61it/s]


Loss: 0.928
Accuracy: 75.36 %
R-Drop Model 0.4 saved at epoch 30 to rdrop_model_0.4_checkpoint.pth

--- Finished R-Drop Model 0.4 Training ---



In [ ]:
# R-Drop Training (alpha=0.6)
run_training(num_epochs=30, learning_rate=0.001, alpha=0.6)

--- Starting R-Drop Model 0.6 Training ---
No saved R-Drop Model 0.6 found. Starting training from scratch.


Epoch 1 [R-Drop Training]: 100%|██████████| 782/782 [02:50<00:00,  4.58it/s]


Loss: 1.782
Accuracy: 50.83 %
R-Drop Model 0.6 saved at epoch 1 to rdrop_model_0.6_checkpoint.pth


Epoch 2 [R-Drop Training]: 100%|██████████| 782/782 [02:52<00:00,  4.52it/s]


Loss: 1.546
Accuracy: 56.20 %
R-Drop Model 0.6 saved at epoch 2 to rdrop_model_0.6_checkpoint.pth


...

Loss: 1.033
Accuracy: 73.08 %
R-Drop Model 0.6 saved at epoch 28 to rdrop_model_0.6_checkpoint.pth


Epoch 29 [R-Drop Training]: 100%|██████████| 782/782 [02:46<00:00,  4.69it/s]


Loss: 1.033
Accuracy: 73.06 %
R-Drop Model 0.6 saved at epoch 29 to rdrop_model_0.6_checkpoint.pth


Epoch 30 [R-Drop Training]: 100%|██████████| 782/782 [02:46<00:00,  4.70it/s]


Loss: 1.021
Accuracy: 72.94 %
R-Drop Model 0.6 saved at epoch 30 to rdrop_model_0.6_checkpoint.pth

--- Finished R-Drop Model 0.6 Training ---



In [ ]:
# R-Drop Training (alpha=0.8)
run_training(num_epochs=30, learning_rate=0.001, alpha=0.8)

--- Starting R-Drop Model 0.8 Training ---
No saved R-Drop Model 0.8 found. Starting training from scratch.


Epoch 1 [R-Drop Training]: 100%|██████████| 782/782 [02:45<00:00,  4.72it/s]


Loss: 1.743
Accuracy: 51.10 %
R-Drop Model 0.8 saved at epoch 1 to rdrop_model_0.8_checkpoint.pth


Epoch 2 [R-Drop Training]: 100%|██████████| 782/782 [02:48<00:00,  4.65it/s]


Loss: 1.501
Accuracy: 58.40 %
R-Drop Model 0.8 saved at epoch 2 to rdrop_model_0.8_checkpoint.pth


...

Loss: nan
Accuracy: 10.00 %
R-Drop Model 0.8 saved at epoch 28 to rdrop_model_0.8_checkpoint.pth


Epoch 29 [R-Drop Training]: 100%|██████████| 782/782 [02:41<00:00,  4.83it/s]


Loss: nan
Accuracy: 10.00 %
R-Drop Model 0.8 saved at epoch 29 to rdrop_model_0.8_checkpoint.pth


Epoch 30 [R-Drop Training]: 100%|██████████| 782/782 [02:41<00:00,  4.84it/s]


Loss: nan
Accuracy: 10.00 %
R-Drop Model 0.8 saved at epoch 30 to rdrop_model_0.8_checkpoint.pth

--- Finished R-Drop Model 0.8 Training ---

